In [190]:
import numpy as np
import pylab as plt
%matplotlib inline

# Datum
trainX = np.loadtxt('X_train.csv', delimiter=',', skiprows=1)
traint = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)
traint = traint[:,1][:,None]
#print("Train X shape:", trainX.shape)
#print("Train t shape:", traint.shape)

def random_classifier():
    return np.random.randint(1,2+1)

def cv(trainX, traint, folds=10, K=2):
    slice_size = int(trainX.shape[0]/folds)
    total_accuracy = 0
    
    for i in range(folds):
        split_start = slice_size * i
        split_end = slice_size * (i+1)
        cv_training_X = np.concatenate((trainX[:split_start], trainX[split_end:]))
        cv_training_t = np.concatenate((traint[:split_start], traint[split_end:]))
        cv_testing_X = trainX[split_start:split_end]
        cv_testing_t = traint[split_start:split_end]
        
        # Logit
        w_nr = get_nr_weights(trainX, traint)
        
        hits = 0
        for j in range(slice_size):
            
            #
            # Pick classifier here
            #
            #c = knn_classifier(cv_training_X, cv_training_t, cv_testing_X[j,:], K)
            c = logit_classifier(w_nr, cv_testing_X[j,:])
            #c = random_classifier()
            
            if (c == cv_testing_t[j]):
                hits += 1
        accuracy = hits/cv_testing_t.shape[0]
        #print("Iteration and accuracy:", i, accuracy)
        total_accuracy += accuracy
    
    total_accuracy /= folds
    return total_accuracy
    

def knn_classifier(trainX, traint, testrow, K=2):
    distances = ((trainX - testrow)**2).sum(axis=1)
    dc = zip(distances,traint)
    dc = sorted(dc,key = lambda x:x[0])
    classes = []
    votes = []
    for k in range(K):
        this_class = dc[k][1] # get the class of the kth ranked
        if not this_class in classes:
            classes.append(this_class)
            votes.append(1)
        else:
            index = classes.index(this_class)
            votes[index] += 1
    best_class = classes[0]
    best_vote = votes[0]
    pos = 1
    for cl in classes[1:]:
        if votes[pos] > best_vote:
            best_vote = votes[pos]
            best_class = cl
        pos += 1
    return best_class

def main_knn():
    Ks = np.arange(1,10,1)
    for k in Ks:
        accuracy = cv(trainX, traint, 10, k)
        print("K and accuracy:", k, accuracy)
        
# Newton Rhapson
def get_nr_weights(trainx, traint, n_its=20):   
    w_count = trainx.shape[1]
    w = np.zeros((w_count,1))
    dx = np.diag(np.dot(trainx, trainx.T))[:,None]
    allw = np.zeros((w_count, n_its))

    for i in range(n_its):
        allw[:,i] = w.flatten()
        P = 1.0/(1.0 + np.exp(-np.dot(trainx, w)))
        gw = -w + np.sum(trainx*np.tile(traint-P, (1,w_count)), axis=0)[:,None]
        temp = trainx*np.tile(P*(1-P), (1,w_count))

        hw = -np.eye(w_count) - np.dot(temp.T,trainx)   
        w = w - np.dot(np.linalg.inv(hw),gw)

    return w

def sigmoid(testrow, w):
    return 1.0/(1.0 + np.exp(-np.dot(testrow, w)))

def logit_classifier(w, testrow):
    c = sigmoid(testrow, w)
    #return c
    if c == 2:
        return 1
    return 2

def main_logit():
    accuracy = cv(trainX, traint, 10)
    print("Accuracy:", accuracy)

# Choose one
#main_knn()
#main_logit()

/home/dom/.local/lib/python3.5/site-packages/ipykernel_launcher.py:89: RuntimeWarning: overflow encountered in exp


Accuracy: 0.6399999999999999


In [162]:
# Submit, lol
w_nr = get_nr_weights(trainX, traint)

testdata = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)
print(testdata.shape)

predictions = np.dot(testdata, w_nr).flatten()
print(predictions.shape)

pred = np.ones((N, 2))
pred[:,0] = range(N)
pred[:,1] = predictions
np.savetxt('pred_classification.csv', pred, fmt='%d', delimiter=",", header="Id,PRP", comments="")

/home/dom/.local/lib/python3.5/site-packages/ipykernel_launcher.py:92: RuntimeWarning: overflow encountered in exp


(1596, 112)
(1596,)
